In [1]:
import pandas as pd
from googletrans import Translator

In [10]:
#import the dataset
df = pd.read_csv("./data/interviews/interviews.csv")
df.head(1)

,Unnamed: 0,file_name,label,text
0,0,BG_Box_AleksandarPovetkin_vs_AnthonyDjoshua_NO,NO,"Аз съм в много добра форма, проведох отличен т..."


In [19]:
df.columns

Index(['Unnamed: 0', 'file_name', 'label', 'text', 'English'], dtype='object')

## Translate the dataset in English

In [12]:
#instantiate the translator and detect the language
translator = Translator()
df['English'] = df['text'].apply(translator.translate, src='bg', dest='en').apply(getattr, args=('text',))

Somehow some lines are translated, but some others are not translated into English:

In [16]:
df.loc[20,'English'][:100]

'Тренировъчният лагер почти беше свършил, оставаше ми само още една седмица спаринги.\r\nКогато се качи'

In [20]:
from ipyannotate import annotate
def display_record(record):
    display(record[1].drop(['Unnamed: 0', 'file_name', 'label', 'text']))

data = df.iterrows()


annotation = annotate(data, display=display_record)
ok, error, next, back = annotation.toolbar.buttons

In [21]:
display(annotation)

Annotation(canvas=OutputCanvas(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'English    I a…

In [54]:
!mkdir -p data/interviews/manually_translated/bg
df['Manually_Labeled'] = [task.value for task in annotation.tasks]
for index, row in df.loc[df.Manually_Labeled == 0].iterrows():
    with open('data/interviews/manually_translated/bg/interview_'+ str(index) + '.txt', 'w') as f:
        f.write(row['English'])

In [59]:
#we translate those interviews manually and put them in this directory:
!mkdir -p data/interviews/manually_translated/en

#then we change in the dataframe
for index, row in df.loc[df.Manually_Labeled == 0].iterrows():
    with open('data/interviews/manually_translated/en/interview_'+ str(index) + '.txt', 'r') as f:
        df.loc[index, 'English'] = f.read()

In [64]:
df['text_bg'] = df.text
df['text'] = df.English
df.drop('English',1).to_csv("./data/interviews/interviews_en.csv")

## Preprocessing

In [96]:
df = pd.read_csv("./data/interviews/interviews_en.csv")

In [97]:
df = df[['label','text']]
df.head(1)

,label,text
0,NO,"I am in very good shape, I had an excellent tr..."


In [76]:
import re, string, unicodedata
import nltk
# import contractions
# import inflect
# from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
# from stop_words import get_stop_words
from string import ascii_letters, digits, whitespace

import glob
import errno

In [77]:
def tokenize(text):
    words = nltk.word_tokenize(text)
    return words

In [78]:
def is_ascii(word):
    for c in word:
        if c in ascii_letters:
            return True
    return False

In [79]:
def remove_ascii(words):
    """Remove ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if not is_ascii(word):
            new_words.append(word)
    return new_words

In [80]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [81]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [82]:
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = 'число_' + str(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

In [83]:
def remove_numbers(words):
    """Remove all interger occurrences in list of tokenized words"""
    new_words = []
    for word in words:
        if not word.isdigit():
            new_words.append(word)
    return new_words

In [84]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in get_stop_words('bg'):
            new_words.append(word)
    return new_words

In [85]:
def stem_words(words=None):
    """Stem words in list of tokenized words"""
    print("USE PRESLAV NAKOV's STEMMER !!!")

In [86]:
def remove_empty_words(words):
    new_words = []
    for word in words:
        if word.strip():
            new_words.append(word)
    return new_words

In [87]:
def print_words(df):
    for i, words in enumerate(df['words'], 1):
        print('Interview ' + str(i))
        print(words)

In [88]:
df['words'] = [tokenize(text) for text in df['text']]
print_words(df)

KeyError: 'text'